# Import

In [13]:
import numpy as np

# step16:複雑な計算グラフ
逆伝播の時に深さ優先で探査していたのを、世代という概念を取り入れて改善する。<br>
<img src="../images/世代.png" width="600"> <br>

In [14]:
# DeZeroで使用する変数クラス
class Variable:
    def __init__(self, data):
        # dataがndarray以外だったらエラーが出るようにする
        if data is not None:
            if not isinstance(data,np.ndarray):
                raise TypeError('{} is not suppported'.format(type(data)))
        self.data = data  #　通常の値
        self.grad=None    #　dataに対応した微分値
        self.creator=None #このクラスの変数が生成される関数　例：y=f(x)のとき　yにとってfがcreator
        self.generation=0 # どの世代に属するか

    # creatorのsetter
    def set_creator(self,func):
        self.creator=func
        self.generation=func.generation+1 #生成されるVariableはcreaterよりも１つ高い世代になる

    # 逆伝播を行う関数(ループを使った実装)
    def backward(self):
        # gradがNoneの場合、自動で微分を生成する
        # ones_like：既存の配列と同じシェイプで，すべての要素の値が 1 の配列を生成
        if self.grad is None:
            self.grad=np.ones_like(self.data)

        funcs=[]
        seen_set=set()
        # DeZeroの関数を登録するときに使用
        # 世代の値順に並び替えられる
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)
        while funcs:
            f=funcs.pop() # 関数を取得
            gys=[output.grad for output in f.outputs] # 出力変数がタプルで保存されているのでリストにまとめる
            gxs=f.backward(*gys) # リストを展開して関数fの逆伝播を呼び出す
            if not isinstance(gxs,tuple):
                gxs=(gxs,)
            # 伝播する微分値をVariableのインスタンス変数gradに設定する
            # zip()は複数のイテラブルオブジェクト（リストやタプルなど）の要素をまとめる関数
            for x,gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad+=gx
                # 関数fの入力のうち別の関数から生成された変数があれば微分値を計算する必要があるのでstackに追加する
                # backpropは深さ優先
                if x.creator is not None:
                    add_func(x.creator) # 1つ目の関数をリストに追加
    # 微分をリセットする
    def cleargrad(self):
        self.grad=None

# Variableクラスを処理する関数を定義するクラス
# このクラスを基底クラスとして、共通する機能を実現
class Function:
    # *を変数につけることで可変長引数にする
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]  #　データを取り出す
        ys = self.forward(*xs) #　実際の計算
        # 複数の値の入力に対応
        if not isinstance(ys,tuple):
            ys=(ys,)
        outputs = [Variable(as_array(y)) for y in ys]# Variableとして返す
        
        self.generation=max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self) # 出力変数に生みの親を覚えさせる
        self.inputs=inputs #　入力された変数を覚える
        self.outputs=outputs #　出力も覚えさせる

        #リストの要素が1つの時は最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]

    #順伝播を行う機能
    def forward(self,x):
        raise NotImplementedError() #意図的に例外を発生させる
    # 微分の罫線を行う逆伝播の機能
    def backward(self,gy):
        raise NotImplementedError()

# 足し算を行うクラス
class Add(Function):
    def forward(self, x0,x1):
        y=x0+x1
        return y
    def backward(self, gy):
        return gy,gy


In [15]:
#function classを継承して利便性向上
class Square(Function):
    def forward(self, x):
        return x**2
    #y=x^2の微分を計算dy/dx=2x
    def backward(self,gy):
        x=self.inputs[0].data
        gx=2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y=np.exp(x)
        return y
    def backward(self,gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx
def square(x):
    f=Square()
    return f(x)
def exp(x):
    f=Exp()
    return f(x)
def add(x0,x1):
    return Add()(x0,x1)
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [16]:
x=Variable(np.array(2.0))
a=square(x=)
y=add(square(a),square(a))
y.backward()
print(y.data)
print(x.grad)



32.0
64.0
